# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Описание данных**
- children — количество детей в семье  
- days_employed — трудовой стаж в днях
- dob_days — возраст клиента в годах
- education — образование клиента
- education_id — идентификатор образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — доход в месяц
- purpose — цель получения кредита

## Импорт файла с данными. Общая информация

In [1]:
#имортируем библиотеку pandas
import pandas as pd 
#Импортируем pymystem3 для лемматизации
from pymystem3 import Mystem
m = Mystem()
#Библиотека для подсчета лемм
from collections import Counter
#импорт библиотеки вычислений
import math
import numpy as np

In [2]:
data = pd.read_csv('/datasets/data.csv') #Загружаем данные

In [3]:
data.info() #общая информация о датафрейме

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
data #просмотр переменной

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


In [5]:
data.describe() #Проверка описательной статистики

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


**Вывод**

В таблице 12 колонок и 21525 строк. В 2-х колонках есть пропущенные значения. В days_employed тип данных float вместо int. В колонке total_income нет необходимости в такой точности данных, нужно перевести в int. Есть отрицательные значения в колонке дети, там же есть выбросы, которые необходимо обработать.

## Предобработка данных

### Обработка пропусков

В колонках total_income и days_employed одинаковое количество пропусков. Проверим, что пропущенные значения сотвествуют одним и тем же людям.

In [6]:
data['total_income'].isna().sum() #подсчет пропущеных значений в колонке total_income

2174

In [7]:
#Проверим что всем пропущенным значениям в total_income соответствуют все пропущенные значения в days_employed
len(data[(data['total_income'].isna() == True) & (data['days_employed'].isna() == True)]) 

2174

Возможно эти пропуски отображают людей без работы и заработка. 2174 пропущенных значения - это около 10% об всей выборке, поэтому заполнение нулями или удаление этих строк будет некорректным. Проверим у каких категорий работников пропущенны значения:

In [8]:
data[data['total_income'].isna()]['income_type'].value_counts()

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

Все из этих категорий скорее всего получают доход. Поэтому заполним пропущенные значения в каждой категории соответствующим медианным значением:

In [9]:
income_type_category_median =  data['total_income'].groupby(data['income_type']).median().astype(int)
income_type_category_median

income_type
безработный        131339
в декрете           53829
госслужащий        150447
компаньон          172357
пенсионер          118514
предприниматель    499163
сотрудник          142594
студент             98201
Name: total_income, dtype: int64

In [10]:
#data['total_income'] = data['total_income'].fillna('none')
#for i in range(len(data['total_income'])):
#    if data['total_income'][i] == 'none':
#        data['total_income'][i] = income_type_category_median[data['income_type'][i]]

In [11]:
# для каждого типа занятости из серии, составленной выше
for income_type in income_type_category_median.index: 
# в нашем датафрейме мы выбираем только те строки, где тип занятости соответствует выбранному выше (по очереди), и берем из доход
    data.loc[data['income_type'] == income_type,'total_income'] =  (data
    # и приравниваем их к самим себе (сохраняем), только заполняя пропуски доходом из таблички со средним доходом
    .loc[data['income_type'] == income_type,'total_income'].fillna(income_type_category_median.loc[income_type]))   

In [12]:
data['total_income'].value_counts().sort_values()

64370.933608        1
233632.159449       1
162416.137763       1
122140.327609       1
175780.973093       1
                 ... 
303202.200463       1
150447.000000     147
118514.000000     413
172357.000000     508
142594.000000    1105
Name: total_income, Length: 19356, dtype: int64

Пропущенные значения в колонке days_employed заполним 0, т.к. она не влияет на результаты исследования

In [13]:
data['days_employed'] = data['days_employed'].fillna(0) #Заполнение значений NaN нулями

Проверим отсутствие пропущенных значений.

In [14]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [15]:
type(data['total_income'][0])

numpy.float64

**Вывод**

Теперь все 21525 столбцов заполнены.

### Замена типа данных

В days_employed тип данных float вместо int. В колонке total_income нет необходимости в такой точности данных, нужно перевести её тип в int.

In [16]:
#применяем метод astype(), т.к. с ним можно перевести данные в нужный тип
data['days_employed'] = data['days_employed'].astype(int) 

In [17]:
#применяем метод astype(), т.к. с ним можно перевести данные в нужный тип
data['total_income'] = data['total_income'].astype(int)

In [18]:
#проверка изменения типа данных
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


В колонке days_employed есть отрицательные данные. Это может означать количество дней прошедших с последнего дня официальной работы.

In [19]:
data['days_employed']

0         -8437
1         -4024
2         -5623
3         -4124
4        340266
          ...  
21520     -4529
21521    343937
21522     -2113
21523     -3112
21524     -1984
Name: days_employed, Length: 21525, dtype: int64

In [20]:
#Проверим есть ли заёмщики с отрицательным доходом 
data[data['total_income'] < 0].count()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [21]:
#Проверим есть ли заёмщики с нулевым доходом и отрицательным количеством дней в найме
data[(data['days_employed'] < 0) & (data['total_income'] == 0)].count()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Выявим категории работников с положительным доходом, но отрицательными днями работы.

In [22]:
data[(data['days_employed'] < 0) & (data['total_income'] > 0)]['income_type'].value_counts()

сотрудник          10014
компаньон           4577
госслужащий         1312
в декрете              1
студент                1
предприниматель        1
Name: income_type, dtype: int64

1. Для категорий: *студент*, *в декрете*, *предприниматель*, *компаньон* отрицательное число в days_emploed может означать количество дней прошедших с последней работы по найму.
2. Для *госслужащих* и *сотрудников* это может означать опечатку, так как при этом их доход не нулевой.
3. Также для *сотрудников* это может означать отсутствие подтверждение работы по найму. Но т.к. такие люди составляют почти половину датасета, допустим что это все же опечатека.

Все случаи позволяют нам заменить отрицательные значения на положительные, допуская, что это опечатка или особенности выгрузки данных.

In [23]:
data['days_employed'][data['days_employed'] < 0]

0       -8437
1       -4024
2       -5623
3       -4124
5        -926
         ... 
21519   -2351
21520   -4529
21522   -2113
21523   -3112
21524   -1984
Name: days_employed, Length: 15906, dtype: int64

In [24]:
# Перевод отрицательных значений в положительные
for i in range(len(data)):
    if data.loc[i, 'days_employed'] < 0:
        data.loc[i, 'days_employed'] *= (-1)

In [25]:
# Проверка исправлений
data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем
21521,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем
21522,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость
21523,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля


Проверим значения в столбце children:

In [26]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

47 строк с отрицательным значением. Допускаем, что это опечатка, т.к. количество не значительное в масштабе данных. Заменим на положительное значение.

In [27]:
for i in range(len(data)):
    if data.loc[i, 'children'] < 0:
        data.loc[i, 'children'] *= (-1)

In [28]:
data['children'].value_counts()

0     14149
1      4865
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64

Присутствует также выброс в с количеством детей 20. Скорее всего это опечатка с лишним нулем. Заменим эти значения на 2.

In [29]:
data['children'] = data['children'].replace(20, 2)

**Вывод**

Тип данных успешно изменен с float на int. Отрицательные значения заменены на положительные.

### Обработка дубликатов

In [30]:
#Приведем все строки к нижнему регистру
columns_all = data.columns #Создаём список колонок
columns_str = []
for column in columns_all:
    if data[column].dtype == object:
        columns_str.append(column)

#Создаем цикл перебора колонок, который уменьшает регистр символов, если колонка имеет тип object
for column in columns_str:
    data[column] = data[column].str.lower()

In [31]:
#Посчитаем явные дубликаты
data.duplicated().sum()

71

In [32]:
#Удалим все явные дубликаты с обнулением индексов
data = data.drop_duplicates().reset_index(drop=True)

In [33]:
#Проверим удаление явных дубликатов
data.duplicated().sum()

0

Явные дубликаты удалены. Теперь выявим неявные дубликаты в строковых данных. Проверем уникальные данные в каждой текстовой колонке, чтобы выявить аномалии.

In [34]:
#Методом отобразим и посчитаем уникальные значения в столбце "образование" 
data[columns_str[0]].value_counts()

среднее                15172
высшее                  5250
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [35]:
#Методом отобразим и посчитаем уникальные значения в столбце "семейное положение"  
data[columns_str[1]].value_counts()

женат / замужем          12339
гражданский брак          4151
не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

In [36]:
#Методом отобразим и посчитаем уникальные значения в столбце "пол" 
data[columns_str[2]].value_counts()

f      14174
m       7279
xna        1
Name: gender, dtype: int64

In [37]:
#Методом отобразим и посчитаем уникальные значения в столбце "образование" 
data[columns_str[3]].value_counts()

сотрудник          11084
компаньон           5078
пенсионер           3829
госслужащий         1457
безработный            2
предприниматель        2
в декрете              1
студент                1
Name: income_type, dtype: int64

In [38]:
#Методом отобразим и посчитаем уникальные значения в столбце "цель" 
data[columns_str[4]].value_counts()

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

В столбце 'gender' один человек имеет непонятный пол. Т.к. пол не влияет на результаты исследования, заменим это значение на '' 

In [39]:
data.loc[data['gender'] == 'xna'].index

Int64Index([10684], dtype='int64')

In [40]:
data['gender'].value_counts()

f      14174
m       7279
xna        1
Name: gender, dtype: int64

In [41]:
data = data.drop(data.loc[data['gender'] == 'xna'].index).reset_index(drop=True)

In [42]:
# Проверим результат
data.loc[10682:10686, :]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10682,0,5777,53,среднее,1,не женат / не замужем,4,f,сотрудник,0,153948,покупка недвижимости
10683,1,512,37,высшее,0,женат / замужем,0,f,компаньон,0,174396,ремонт жилью
10684,0,388606,60,среднее,1,гражданский брак,1,f,пенсионер,0,108125,свадьба
10685,0,3619,35,среднее,1,женат / замужем,0,f,сотрудник,0,89817,получение образования
10686,0,921,57,среднее,1,женат / замужем,0,f,сотрудник,0,240825,на покупку подержанного автомобиля


**Вывод**

Явные дубликаты и пропуск в столбце "пол" устранены. Для устранения дубликатов в столбце "цель" необходимо проветсти лемматизацию.

### Лемматизация

Проверим сколько различных целей кредита в текущем датасете

In [43]:
data['purpose'].value_counts().count()

38

In [44]:
# лемматизируем каждую строку в столбце purpose
a1 = [] 
for row in data['purpose']:
    a2 = []
    a2 = ''.join(m.lemmatize(row)).replace('\n', '') # лемматизация каждой строки, удаление лишних пробелов и пропусков строки
    a1.append(a2) # создаем список строк
data['purpose_lemmatized'] = a1 # создаем новый столбец из лемматезированных строк
    

In [45]:
# проверка результата
data.loc[:, 'purpose':'purpose_lemmatized']

,purpose,purpose_lemmatized
0,покупка жилья,покупка жилье
1,приобретение автомобиля,приобретение автомобиль
2,покупка жилья,покупка жилье
3,дополнительное образование,дополнительный образование
4,сыграть свадьбу,сыграть свадьба
...,...,...
21448,операции с жильем,операция с жилье
21449,сделка с автомобилем,сделка с автомобиль
21450,недвижимость,недвижимость
21451,на покупку своего автомобиля,на покупка свой автомобиль


Проверим какие различные целеи кредита осталось после лемматизации.

In [46]:
# вывод уникальных категорий
data['purpose_lemmatized'].value_counts()

автомобиль                                972
свадьба                                   791
на проведение свадьба                     768
сыграть свадьба                           765
операция с недвижимость                   675
покупка коммерческий недвижимость         661
операция с жилье                          652
покупка жилье для сдача                   651
операция с коммерческий недвижимость      650
жилье                                     646
покупка жилье                             646
покупка жилье для семья                   638
строительство собственный недвижимость    635
недвижимость                              633
операция со свой недвижимость             627
строительство жилой недвижимость          624
покупка свой жилье                        620
покупка недвижимость                      620
строительство недвижимость                619
ремонт жилье                              607
покупка жилой недвижимость                606
на покупка свой автомобиль        

In [47]:
lemmas_count = sorted(Counter(data['purpose_lemmatized'].apply(m.lemmatize).sum()).items(), key=lambda x: -x[1])
lemmas_count

[(' ', 33569),
 ('\n', 21453),
 ('недвижимость', 6350),
 ('покупка', 5896),
 ('жилье', 4460),
 ('автомобиль', 4306),
 ('образование', 4013),
 ('с', 2918),
 ('операция', 2604),
 ('свадьба', 2324),
 ('свой', 2230),
 ('на', 2222),
 ('строительство', 1878),
 ('высокий', 1374),
 ('получение', 1314),
 ('коммерческий', 1311),
 ('для', 1289),
 ('жилой', 1230),
 ('сделка', 941),
 ('дополнительный', 906),
 ('заниматься', 904),
 ('проведение', 768),
 ('сыграть', 765),
 ('сдача', 651),
 ('семья', 638),
 ('собственный', 635),
 ('со', 627),
 ('ремонт', 607),
 ('подержанный', 486),
 ('подержать', 478),
 ('приобретение', 461),
 ('профильный', 436)]

Самые популярные леммы: Недвижимость, жилье, автомобиль, образование, свадьба. На основе этого результата выполним категоризацию.

**Вывод**

После лемматизации целей кредита вместо 38 целей осталось37.

### Категоризация данных

Проведем категоризацию в соответствии с целями исследования.

In [48]:
data.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmatized
0,1,8437,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875,покупка жилья,покупка жилье
1,1,4024,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,приобретение автомобиля,приобретение автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885,покупка жилья,покупка жилье
3,3,4124,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628,дополнительное образование,дополнительный образование
4,0,340266,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616,сыграть свадьбу,сыграть свадьба


In [49]:
type(data)

pandas.core.frame.DataFrame

#### Категоризация целей кредита

In [50]:
data['purpose_category'] = data['purpose_lemmatized'] #создаем колонку с категориями целей кредита
for row in data['purpose_category']:
    if 'автомобиль' in row:
        data['purpose_category'] = data['purpose_category'].replace(row, 'покупка автомобиля')
    if 'жилье' in row or 'недвижимость' in row:
        data['purpose_category'] = data['purpose_category'].replace(row, 'покупка/строительство недвижимости')
    if 'свадьба' in row:
        data['purpose_category'] = data['purpose_category'].replace(row, 'свадьба')
    if 'образование' in row:
        data['purpose_category'] = data['purpose_category'].replace(row, 'образование')

In [51]:
data['purpose_category'].value_counts()

покупка/строительство недвижимости    10810
покупка автомобиля                     4306
образование                            4013
свадьба                                2324
Name: purpose_category, dtype: int64

#### Категоризация по наличию детей

In [52]:
data['children_category'] = data['children']

In [53]:
data['children_category'] = data['children_category'].replace(data['children_category'][data['children_category'] >0], 1)

In [54]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmatized,purpose_category,children_category
0,1,8437,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875,покупка жилья,покупка жилье,покупка/строительство недвижимости,1
1,1,4024,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,приобретение автомобиля,приобретение автомобиль,покупка автомобиля,1
2,0,5623,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885,покупка жилья,покупка жилье,покупка/строительство недвижимости,0
3,3,4124,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628,дополнительное образование,дополнительный образование,образование,1
4,0,340266,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616,сыграть свадьбу,сыграть свадьба,свадьба,0


#### Категоризация по возврату кредита в срок

Уже проведена.

#### Категоризация по уровню дохода

Поделим клиентов на 4 группы:
1) Без заработка
2) Средний доход
3) Доход ниже среднего
4) Доход выше среднего 

##### Найдем основные характеристики зарплаты

In [55]:
data['income_category'] = data['total_income'] # добавляем колонку с категорииями дохода

Найдем среднюю зарплату:

In [56]:
income_mean = data['total_income'].mean().astype(int)

Найдем медианную зарплату: 

In [57]:
income_median = data['total_income'].median().astype(int)

Найдем максимальную зарплату:

In [58]:
income_max = data['total_income'].max().astype(int)

Найдем минимальную зарплату:

In [59]:
income_min = data['total_income'].min().astype(int)

Создадим сортированный по возрастанию список доходов:

In [60]:
income_sort = data['total_income'].sort_values(ascending = True).reset_index(drop=True)
income_sort_count = income_sort.count()

Найдем как распределяется доход среди клиентов порядку заработка:

In [61]:
income_degree = income_sort.apply(math.log10).astype(int).value_counts()
income_degree

5    16965
4     4463
6       25
Name: total_income, dtype: int64

In [62]:
print(income_degree[4], 'клиентов зарабатывает меньше 100 тыс. руб.')
print(income_degree[5], 'клиентов зарабатывает от 100 до 999 тыс. руб.')
print(income_degree[6], 'клиентов зарабатывает больше 1 млн.руб')

4463 клиентов зарабатывает меньше 100 тыс. руб.
16965 клиентов зарабатывает от 100 до 999 тыс. руб.
25 клиентов зарабатывает больше 1 млн.руб


##### Присвоим соответствующие категории

Поделим сортированный список на четыре равные группы:

In [63]:
data['income_category'] = pd.qcut(data.total_income, 4, labels=["низкий доход", "ниже среднего", "выше среднего", "высокий"])

In [64]:
#проверка результата
data['income_category'].value_counts()

ниже среднего    5479
низкий доход     5364
высокий          5363
выше среднего    5247
Name: income_category, dtype: int64

**Вывод**

Выделены категории данных там где это необходимо для ответов на вопросы. Доход и цели кредита поделены на 4 группы.

## Ответы на вопросы исследования

### Есть ли зависимость между наличием детей и возвратом кредита в срок?

Выведем сводную таблицу показывающую количество клиентов в каждой категории:

In [65]:
#data.groupby(['children_category', 'debt']).count()
child_debt_df = data.pivot_table('children', index='children_category', columns='debt', aggfunc='count')
child_debt_df.columns = ['no_debt', 'have_debt'] 
child_debt_df['percent'] = child_debt_df['have_debt'] / (child_debt_df['no_debt'] + child_debt_df['have_debt'])
child_debt_df['percent'] = child_debt_df['percent'].apply(lambda x: f'{x:.1%}')

In [66]:
child_debt_df.sort_values('percent')

,no_debt,have_debt,percent
children_category,,,
0,13027,1063,7.5%
1,6685,678,9.2%


**Вывод**

1) Среди клиентов без детей вернувших кредит в срок почти в 12 раз больше чем не вернувших.  
2) Среди клиентов с детьми вернувших кредит в срок почти в 10 раз больше чем не вернувших.  
Наличие детей в семье **незначительно влияет** на факт возврата кредита в срок. Клиенты с детьми реже возвращают кредит в срок.

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [1]:
famili_status_debt = data.pivot_table('children', index='family_status', columns='debt', aggfunc='count')
famili_status_debt.columns = ['no_debt', 'have_debt'] 
famili_status_debt['percent'] = famili_status_debt['have_debt'] / (famili_status_debt['no_debt'] + famili_status_debt['have_debt'])
famili_status_debt['percent'] = famili_status_debt['percent'].apply(lambda x: f'{x:.1%}')

NameError: name 'data' is not defined

In [68]:
famili_status_debt.sort_values('percent')

,no_debt,have_debt,percent
family_status,,,
вдовец / вдова,896,63,6.6%
в разводе,1110,85,7.1%
женат / замужем,11408,931,7.5%
гражданский брак,3762,388,9.3%
не женат / не замужем,2536,274,9.8%


**Вывод**

1) Чаще всего задерживают кредиты люди не в браке также как и люди в гражданском браке.  
2) Реже задерживают возврат в срок клиенты в браке и разведенные.  
3) Меньше всех задержек в возврате среди овдовевших.

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [69]:
income_debt = data.pivot_table('children', index='income_category', columns='debt', aggfunc='count')
income_debt.columns = ['no_debt', 'have_debt'] 
income_debt['percent'] = income_debt['have_debt'] / (income_debt['no_debt'] + income_debt['have_debt'])
income_debt['percent'] = income_debt['percent'].apply(lambda x: f'{x:.1%}')

In [70]:
income_debt.sort_values('percent')

,no_debt,have_debt,percent
income_category,,,
высокий,4980,383,7.1%
низкий доход,4937,427,8.0%
выше среднего,4799,448,8.5%
ниже среднего,4996,483,8.8%


**Вывод**

1) Клиенты с высоким заработком реже всех задерживают выплату по кредиту.  
2) Клиенты с заработком ниже среднего чаще всех задерживают выплату по кредиту.  

### Как разные цели кредита влияют на его возврат в срок?

In [71]:
purpose_debt = data.pivot_table('children', index='purpose_category', columns='debt', aggfunc='count')
purpose_debt.columns = ['no_debt', 'have_debt'] 
purpose_debt['percent'] = purpose_debt['have_debt'] / (purpose_debt['no_debt'] + purpose_debt['have_debt'])
purpose_debt['percent'] = purpose_debt['percent'].apply(lambda x: f'{x:.1%}')

In [72]:
purpose_debt.sort_values('percent')

,no_debt,have_debt,percent
purpose_category,,,
покупка/строительство недвижимости,10028,782,7.2%
свадьба,2138,186,8.0%
образование,3643,370,9.2%
покупка автомобиля,3903,403,9.4%


**Вывод**

1) Чаще всего задерживают кредиты клиенты, которые берут кредит на автомобиль.  
2) Чуть лучше ситуация с клиентами, которые кредитуются для получения образования.  
3) Лучще всех возвращают кредиты клиенты, которые берут кредит для покупки или строительства недвижимости.  
4) На 2-ом месте по возвратам - кредиты на проведение свадьбы.

## Общий вывод

1. Самый **надежный заёмщик** клиент с заработком выше среднего, овдовевший или разведенный, без детей кредитующийся для покупки или строительства недвижимости.  
2. **Наименее надежный** - заемщик со средним заработком, не в браке или в гражданском браке, с детьми, кредитующийся на покупку автомобиля или для оплаты образования.  